# Dumping


In [1]:
import pickle
from ZV_functions import *
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, GridSearchCV
from xgboost.sklearn import XGBClassifier 
from sklearn import metrics
from sklearn.utils.class_weight import compute_class_weight


#base_dir='/eos/home-a/ahakimi/www/ZV_analysis'
base_dir='D:\\Travail\\ZV_analysis\\'
#year='2018_SR'
years=['2018_nobtag_var', '2017_nobtag_var', '2016_nobtag_var']
name='All_years_nobtag_allfeats'
cut='Resolved_SR'
vers=''


base_dir='D:\\Travail\\ZV_analysis\\'
model_name='DNN_220_220_220_220_0_lr0.0001_dp0.1'
model_dir=base_dir+name+'\\'+cut+'\\Res_{}'.format(model_name)
vers=''
X,y, variables_list=prep_data_multi(years,name, cut, vers, base_dir)

if cut == 'Boosted_SR':
    features =['Lep_pt1', 'Lep_pt2', 'Lep_eta1',
           'Lep_eta2', 'mll',
           'FatJet_pt', 'FatJeteta',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'mjj_max', 'detajj_mjjmax', 'dphijj_mjjmax', 
            'njet', 'Zvjet', 'vbs_jet_qgl1', 'vbs_jet_qgl2', 'nbtag'
              ]
elif cut == 'Resolved_SR':
    features=['Lep_pt1', 'Lep_pt2', 'Lep_eta1',
           'Lep_eta2', 'mll',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'V_jet_pt1', 'V_jet_pt2',
           'V_jet_eta1', 'V_jet_eta2',
           'mjj_max', 'detajj_mjjmax', 'dphijj_mjjmax',
           'V_jet_mass', 'njet', 'vbs_jet_qgl1', 'vbs_jet_qgl2',
             'V_jet_qgl1', 'V_jet_qgl2', 'nbtag']
input_list= features
inputs= [cut+'_'+i for i in input_list]

## SAVE THE MODEL, ITS METADATA AND TRAINING INFORMATIONS

# dump the variables list
import yaml
varfile = os.path.join(model_dir, "variables.yml")
old_file= os.path.join(model_dir, "variables_old.yml")
if os.path.isfile(varfile):
    print("ACHTUNG! variables file already existing: old file renamed with '_old'")
    if os.path.isfile(old_file):
        os.remove(old_file)
    os.rename(varfile, varfile[:-4] + "_old.yml")
with open(varfile, "w") as out_var_file:
    out_var_file.write(yaml.dump(inputs))
    

## Export:
## * keras model to tensorflow model
## * tf metadata
## * scaler

args = {
    "dir": model_dir,
    "input": "model.h5",
    "output": "model.pb",
    "tf_metadata": "tf_metadata.txt",
    "input_scaler": "scaler_model.pkl",
    "output_scaler": "scaler.txt",
}
from keras import backend as K
import tensorflow as tf

# This line must be executed before loading Keras model.
K.set_learning_phase(0)
auc=tf.keras.metrics.AUC()      
from keras.models import load_model
model = load_model(os.path.join(args["dir"], args["input"]))


def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, args["dir"], args["output"], as_text=False)

## save tensorflow metadata
with open(os.path.join(args["dir"], args["tf_metadata"]), "w") as f:
    f.write(str(model.inputs[0].name) + " " + str(model.outputs[0].name) + "\n")

2018_nobtag_var
2017_nobtag_var
2016_nobtag_var
Resolved_SR_Lep_pt1 114.884834 83.541824
Resolved_SR_Lep_pt2 50.74951 33.16918
Resolved_SR_Lep_eta1 -0.002300476 1.1978709
Resolved_SR_Lep_eta2 -0.0027607393 1.2011259
Resolved_SR_mll 90.81847 4.419396
Resolved_SR_Zepp_ll 0.64564437 0.5090893
Resolved_SR_nFatJet 1.0888253911071453 1.0804754553350302
Resolved_SR_FatJet_pt -999.0 0.0
Resolved_SR_FatJeteta -999.0 0.0
Resolved_SR_FatJet_softdropmass 0.0 0.0
Resolved_SR_FatJet_tau21 -999.0 0.0
Resolved_SR_Zlep_1 -0.00066896365 0.44106093
Resolved_SR_Zlep_2 -0.0007420545 0.44197175
Resolved_SR_category 1.0 0.0
Resolved_SR_vbs_jet_pt1 176.62933 128.44662
Resolved_SR_vbs_jet_pt2 80.11245 67.165306
Resolved_SR_vbs_jet_eta1 -0.0012387873 1.8911467
Resolved_SR_vbs_jet_eta2 0.0010057766 2.4798346
Resolved_SR_V_jet_pt1 81.620346 52.423412
Resolved_SR_V_jet_pt2 43.014473 14.131648
Resolved_SR_V_jet_eta1 -0.000108597626 1.2766782
Resolved_SR_V_jet_eta2 -0.0004988865 1.3612747
Resolved_SR_mjj_max 779.600

D:\Travail\ZV_analysis\ZV_functions.py:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['signal']=y['signal'].astype('int32')
D:\Travail\ZV_analysis\ZV_functions.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['w']=0
D:\Travail\ZV_analysis\ZV_functions.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[

saving files
Data prepared, X and y saved to D:\Travail\ZV_analysis\All_years_nobtag_allfeats\Resolved_SR\data.


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 85 variables.
INFO:tensorflow:Converted 85 variables to const ops.


In [2]:
#dump scaler
#be careful to but the scaler pickle in the DNN folder

scaler = pickle.load(open(os.path.join(args["dir"], args["input_scaler"]), 'rb'))
with open(os.path.join(args["dir"], args["output_scaler"]), "w") as f:
    scaler_df=pd.DataFrame(zip(scaler.mean_, scaler.scale_),index=variables_list).T
    for input in inputs:
        mean = scaler_df[input][0]
        scale = scaler_df[input][1]
        if cut=='Resolved_SR':
            var=input[12:]
        elif cut=='Boosted_SR':
            var=input[11:]
        print(var, mean, scale)
        #print('var:{}, type: {}'.format(var, type(mean.astype('float32'))))
        
        f.write(var + " " + str(mean.astype('float32')) + " " + str(scale.astype('float32')) + "\n")
        
print("Dumping done.")

Lep_pt1 114.88488741425083 83.54180390980069
Lep_pt2 50.749511365695646 33.16917368340351
Lep_eta1 -0.0023004773196994423 1.1978705955861233
Lep_eta2 -0.002760740289972467 1.2011256344120511
mll 90.81844035459399 4.4193949439895155
Zlep_1 -0.0006689639226749412 0.4410608361648756
Zlep_2 -0.0007420546546934376 0.44197167370084167
vbs_jet_pt1 176.62937935334307 128.44660337562777
vbs_jet_pt2 80.1124715660709 67.16529477871873
vbs_jet_eta1 -0.00123878707789778 1.8911462755775037
vbs_jet_eta2 0.0010057765033182628 2.4798341660098986
V_jet_pt1 81.62034972014929 52.42340151876261
V_jet_pt2 43.01446431614098 14.131645404567438
V_jet_eta1 -0.00010859751765264402 1.27667791840229
V_jet_eta2 -0.0004988865528195821 1.361274408784105
mjj_max 779.6006904928355 544.3881349754718
detajj_mjjmax 3.7686626817058637 1.2038706843800087
dphijj_mjjmax 1.9456443992223849 0.8806426746619741
V_jet_mass 85.30872663650945 10.130172418740223
njet 5.071696463322559 1.0995541913393974
vbs_jet_qgl1 0.526035149512499